In [13]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


import datetime
import glob


import nltk
import string, re

from sklearn.feature_extraction.text import TfidfVectorizer



stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wl = nltk.stem.WordNetLemmatizer()



In [14]:
covid_words = open("data/covid_words.txt").read().split()
religious_words = list(pd.read_csv('data/religious_words_final_df.csv').religious_words)



###########################################################
# df = pd.DataFrame()
# for fname in glob.glob(r"data/twitter_data/*.csv"):
#     day_df=pd.read_csv(fname)
#     day_df['date']= datetime.date(2020, 5, int(fname.partition("May")[2][:-4]))
#     df=df.append(day_df)
#     # print(fname, fname.partition("May")[2][:-4], len(df))

# df.to_csv('data/twitter_data/covid_data_full.csv', index=false)
###########################################################




###########################################################
# f = open('data/religious_final_words.txt', 'w')
# f.write(' '.join(sorted(religious_words)))
# f.close()
###########################################################

In [19]:
df = pd.read_csv('data/twitter_data/covid_data_full.csv', index_col=False, low_memory=False)
df.head(2)

,Unnamed: 0,id,created_at,original_text,clean_text,sentiment,lang,screen_name,location,date
0,0,1264707752217186307,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,RT morethanmySLE Dr. Birx leaves American peop...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,Backstorymom1,NaN,2020-05-24
1,1,1264707752032636929,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,RT KaitSnod There Corona Virus America https /...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,SWafflez_,At the link down below 👀,2020-05-24


In [46]:
def clean_twitter(text):
    text = "".join([char.lower() for char in text if char not in string.punctuation])
    tokens = re.split('\W+', text)
    # text = [ps.stem(wl.lemmatize(word)) for word in tokens if word not in stopwords]
    text = [word for word in tokens if word not in stopwords]
    for word in text.copy():
        text.append(wl.lemmatize(word))
        text.append(ps.stem(word))
        
    return " ".join(list(set(text)))

# clean_twitter("D@on't do that ! runn@er running!!!")

In [47]:
df['clean_text_old'] = df['clean_text']
df['clean_text'] = df['original_text'].apply(lambda x: clean_twitter(x))
df.head(2)

,Unnamed: 0,id,created_at,original_text,clean_text,sentiment,lang,screen_name,location,date,clean_text_old
0,0,1264707752217186307,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,leaf covid19 leav question answers dr people a...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,Backstorymom1,NaN,2020-05-24,RT morethanmySLE Dr. Birx leaves American peop...
1,1,1264707752032636929,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,america corona kaitsnod httpstcoz9ndlkz3ml vir...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,SWafflez_,At the link down below 👀,2020-05-24,RT KaitSnod There Corona Virus America https /...


In [12]:
def find_religious_words(text):
#     return list(set(text.split()) & set(religious_words))
    return list(filter(lambda x: x in text, religious_words))
                
# find_religious_words('allah help me pls, amen! islam&peace')

['allah', 'amen', 'islam']

In [51]:
df['religious_text'] = df['clean_text'].apply(lambda x: find_religious_words(x))
df.head(2)

,Unnamed: 0,id,created_at,original_text,clean_text,sentiment,lang,screen_name,location,date,clean_text_old,religious_text
0,0,1264707752217186307,Sun May 24 23:59:59 +0000 2020,RT @morethanmySLE: Dr. Birx leaves the America...,leaf covid19 leav question answers dr people a...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,Backstorymom1,NaN,2020-05-24,RT morethanmySLE Dr. Birx leaves American peop...,[]
1,1,1264707752032636929,Sun May 24 23:59:59 +0000 2020,RT @KaitSnod: There is no Corona Virus in Amer...,america corona kaitsnod httpstcoz9ndlkz3ml vir...,"Sentiment(polarity=0.0, subjectivity=0.0)",en,SWafflez_,At the link down below 👀,2020-05-24,RT KaitSnod There Corona Virus America https /...,[]


In [52]:
df.to_csv('data/religious_twitter_df.csv', index=False)
# df.to_csv('data/religious_twitter_df.csv', index=False).sort_values('religious_text')

######################### ------------------------------- #########################

In [ ]:
def clean_twitter(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(wl.lemmatize(word)) for word in tokens if word not in stopwords]
    return text

In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

df['text_len'] = df['original_text'].apply(lambda x: len(x) - x.count(" "))
df['punct%'] = df['original_text'].apply(lambda x: count_punct(x))

In [ ]:
df.head(2)

In [ ]:
# tfidf_vect = TfidfVectorizer(analyzer=clean_text)
# tfidf_vect_fit = tfidf_vect.fit(df['original_text'])
# tfidf_data = tfidf_vect_fit.transform(df['original_text'])


# vect_df = pd.concat([df[['text_len', 'punct%']].reset_index(drop=True), pd.DataFrame(tfidf_data.toarray())], axis=1)
# vect_df.head(2)